In [28]:
import numpy as np
import pandas as pd
import lightgbm as lgbm
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import time
from sklearn.linear_model import LinearRegression
from tqdm import tqdm_notebook as tqdm
from tsfresh import extract_features
from lightgbm import LGBMRegressor

In [2]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [3]:
# Загрузка данных
# windows
PATH_TO_DATA = os.path.join('../data')
full_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col='id')
full_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'), index_col='id')

In [4]:
# from sklearn.model_selection import TimeSeriesSplit

# N_FOLDS = 5
# ts_splitter = TimeSeriesSplit(N_FOLDS)

### Важно! В train.csv и test.csv признаки `epoch` и `sat_id` стоят в разном порядке. Это может быть критично для некоторых моделей (например, lgbm). Поэтому сделаем единный порядок:

In [15]:
test_cols = list(full_test.columns)
test_cols[0] = 'epoch'
test_cols[1] = 'sat_id'
full_test = full_test.reindex(columns=test_cols)

In [16]:
full_train['epoch'] = full_train.epoch.apply(lambda x: time.mktime(datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f').timetuple())).values
full_test['epoch'] = full_test.epoch.apply(lambda x: time.mktime(datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f').timetuple())).values

In [17]:
full_train.head()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,,
0,1.388520e+09,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1.388523e+09,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,1.388526e+09,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,1.388528e+09,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,1.388531e+09,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [9]:

# kind_to_fc_parameters = {
#     "x_sim": {"mean": None},#, "max": None, "min": None},
#     "y_sim": {"mean": None},#, "max": None, "min": None},
#     "z_sim": {"mean": None},#, "max": None, "min": None},
#     "Vx_sim": {"mean": None},#, "max": None, "min": None},
#     "Vy_sim": {"mean": None},#, "max": None, "min": None},
#     "Vz_sim": {"mean": None}#, "max": None, "min": None},
    
}

# extracted_features = extract_features(full_train, kind_to_fc_parameters=None,
#                                       column_id="sat_id", column_sort="epoch")

In [18]:
# import pickle

In [19]:
# with open('timeseries_features', 'wb') as handle:
#     pickle.dump(extracted_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# extracted_features

In [21]:
full_train.head()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,,
0,1.388520e+09,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1.388523e+09,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,1.388526e+09,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,1.388528e+09,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,1.388531e+09,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [22]:
# x_num

for i, col in enumerate(['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']):
    full_train[col + '_num'] = 0
    arr = full_train[col + '_num'].values
    start = 0
    for sat_id in range(600):
        df = full_train[full_train.sat_id == sat_id]
        step = df.shape[0]
        idx_min = df[col].idxmin() % 24
        arr[start: start + step] = np.fromfunction(lambda i: (i + idx_min) % 24, (step, ))
        start += step
    full_train[col + '_num'] = arr

In [23]:
# x_num

for i, col in enumerate(['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']):
    full_test[col + '_num'] = 0
    arr = full_test[col + '_num'].values
    start = 0
    for sat_id in full_test.sat_id.unique():
        df = full_test[full_test.sat_id == sat_id]
        step = df.shape[0]
        idx_last = full_train[full_train.sat_id == sat_id][col + '_num'].iloc[-1] + 1
        arr[start: start + step] = np.fromfunction(lambda i: (i + idx_last) % 24, (step, ))
        start += step
    full_test[col + '_num'] = arr
        

In [24]:
for col_name in ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']:
    full_test[col_name] = 0.0

In [26]:
%%time

features_for_tsfresh = ['sat_id', 'epoch', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
target_cols = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

width = 8
for sat_id in tqdm(full_test.sat_id.unique()):
    df1_train = full_train[full_train.sat_id == sat_id]
    df1_test =  full_test[full_test.sat_id == sat_id]
    for col in target_cols:
        for i in range(24):
            df2_train = df1_train[df1_train[col + '_sim_num'] == i]
            df2_test = df1_test[df1_test[col + '_sim_num'] == i]
            
            train_features = extract_features(df2_train[features_for_tsfresh], kind_to_fc_parameters=None,
                                              column_id="sat_id", column_sort="epoch")
            test_features = extract_features(df2_test[features_for_tsfresh], kind_to_fc_parameters=None,
                                              column_id="sat_id", column_sort="epoch")
            tmp = np.zeros((df2_train.shape[0], train_features.size))
            tmp = train_features.values + tmp
            df2_train = pd.concat([df2_train, pd.DataFrame(tmp, index=df2_train.index)], axis=1)
            
            tmp = np.zeros((df2_test.shape[0], test_features.size))
            tmp = test_features.values + tmp
            df2_test = pd.concat([df2_test, pd.DataFrame(tmp, index=df2_test.index)], axis=1)
            
            
            lin_model = LGBMRegressor()
            X_train = df2_train.reset_index().drop(columns=target_cols).values[-width:]
            y_train = df2_train[col].values[-width:]
            lin_model.fit(X_train, y_train)
            X_test = df2_test.reset_index().drop(columns=target_cols).values[-width:]
            y_test = lin_model.predict(X_test)
            full_test.loc[X_test[:, 0].ravel(), col] = y_test

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



NameError: name 'extract_features' is not defined

In [20]:
df2_train

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,...,4514,4515,4516,4517,4518,4519,4520,4521,4522,4523
_temporary_index_column,,,,,,,,,,,,,,,,,,,,,
49187,1.388581e+09,25,50148.144734,6095.977588,25768.405552,-0.223176,0.921858,-1.917990,50130.410855,6126.332443,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49211,1.388667e+09,25,50147.567353,6067.572767,25805.144410,-0.221002,0.922435,-1.917114,50126.538829,6115.274203,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49235,1.388752e+09,25,50148.018441,6038.992785,25841.655839,-0.218733,0.922883,-1.916276,50123.528203,6104.473634,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49259,1.388838e+09,25,50149.412834,6011.059028,25877.662880,-0.216324,0.923260,-1.915454,50121.330135,6094.431299,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49283,1.388924e+09,25,50151.261376,5984.298706,25913.102319,-0.213777,0.923648,-1.914623,50119.697514,6085.444262,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49307,1.389010e+09,25,50152.976126,5958.649188,25948.358951,-0.211128,0.924119,-1.913754,50118.269980,6077.535671,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49331,1.389096e+09,25,50154.039289,5933.587719,25983.993861,-0.208435,0.924712,-1.912830,50116.683107,6070.472053,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49355,1.389181e+09,25,50154.219765,5908.377880,26020.583503,-0.205754,0.925424,-1.911848,50114.662902,6063.850620,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0
49379,1.389264e+09,25,50219.529077,2508.571790,32518.481628,0.156865,0.956589,-1.701318,50112.079143,6057.216328,...,1.0,1.0,-2.649868e+10,-5.313346e+10,-7.970580e+10,0.0,0.0,0.0,8105.278816,1.0


In [21]:
df2_test

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x_sim_num,y_sim_num,...,4514,4515,4516,4517,4518,4519,4520,4521,4522,4523
_temporary_index_column,,,,,,,,,,,,,,,,,,,,,
49931,25,1.391233e+09,50047.578228,5857.688367,25400.943180,-0.229928,0.927619,-1.932110,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
49955,25,1.391319e+09,50046.348709,5849.464732,25388.401057,-0.229974,0.927454,-1.932676,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
49979,25,1.391405e+09,50045.270662,5842.236602,25375.586049,-0.229934,0.927367,-1.933207,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
50003,25,1.391490e+09,50043.988248,5835.727109,25362.906319,-0.229853,0.927385,-1.933695,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
50027,25,1.391576e+09,50042.252345,5829.514590,25350.735850,-0.229776,0.927509,-1.934142,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
50051,25,1.391658e+09,50039.963764,5823.149731,25339.335115,-0.229740,0.927715,-1.934554,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
50075,25,1.391744e+09,50037.169644,5816.256235,25328.812879,-0.229768,0.927968,-1.934943,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
50099,25,1.391830e+09,50034.027099,5808.592258,25319.130175,-0.229862,0.928227,-1.935322,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0
50123,25,1.391916e+09,50030.753345,5800.069830,25310.132498,-0.230012,0.928453,-1.935703,12,16,...,1.0,1.0,-2.576672e+10,-5.114986e+10,-7.631839e+10,0.0,0.0,0.0,6461.359647,1.0


In [19]:
full_test[['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].to_csv('submission.csv', index_label='id')

SystemError: <built-in method item of numpy.ndarray object at 0x7f4d605484e0> returned a result with an error set